In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation
pd.set_option('max_columns',50)

In [19]:
plays = pd.read_csv('data/plays.csv')
data = pd.read_csv('data/all_tracking.csv')

In [8]:
plays.head(10)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE_ROLLOUT_LEFT,0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",TRADITIONAL,0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False
5,2018090600,320,(10:10) (Shotgun) N.Foles pass short left to N...,1,2,8,PHI,play_type_pass,PHI,4,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0.0,0.0,10:10:00,14.0,NaN,NaN,C,4,4,-0.344096,False
6,2018090600,344,(9:24) (Shotgun) N.Foles pass incomplete short...,1,3,4,PHI,play_type_pass,PHI,8,EMPTY,"1 RB, 2 TE, 2 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0.0,0.0,09:24:00,18.0,NaN,NaN,I,0,0,-1.192208,False
7,2018090600,402,(9:08) M.Ryan pass incomplete deep left to M.S...,1,1,10,ATL,play_type_pass,PHI,44,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",SCRAMBLE_ROLLOUT_LEFT,0.0,0.0,09:08:00,54.0,NaN,NaN,I,0,0,-0.429863,False
8,2018090600,492,(7:01) M.Ryan pass short left to T.Coleman pus...,1,2,13,ATL,play_type_pass,PHI,36,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,07:01:00,46.0,NaN,NaN,C,26,26,1.879804,False
9,2018090600,521,(6:19) M.Ryan pass short left to A.Hooper to P...,1,1,10,ATL,play_type_pass,PHI,10,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,3.0,"4 DL, 2 LB, 5 DB",DESIGNED_ROLLOUT_LEFT,0.0,0.0,06:19:00,20.0,NaN,NaN,C,3,3,0.045665,False


In [22]:
def get_playId(plays,gameId):
    return plays[plays['gameId']==gameId]['playId'].values

In [21]:
def get_plays(plays,gameId):
    return plays[plays['gameId']==gameId]

In [25]:
def get_track_game(track,gameId):
    return track[track['gameId']==gameId]

In [26]:
def get_track_play(track,playId):
    return track[track['playId']==playId]

In [23]:
def get_coord(plays,track):
    c_x = {}
    c_y = {}
    for f in set(plays['frameId']):
        c_x[f] = [x for x,x_f in zip(track['x'],track['frameId']) if x_f==f]
        c_y[f] = [y for y,y_f in zip(track['y'],track['frameId']) if y_f==f]
        
    return c_x,c_y

In [24]:
def animate_play(home_coord,away_coord,football_coord,plays):
    anim_fig,anim_ax = plt.subplots(figsize=(15,10))
    img = plt.imread('football_field_grayscale.png')
    anim_ax.imshow(img,extent=[0,120,0,53.34])

    h, = anim_ax.plot(home_coord.iloc[0]['h_x'],home_coord.iloc[0]['h_y'],'ro')
    a, = anim_ax.plot(away_coord.iloc[0]['h_x'],away_coord.iloc[0]['h_y'],'bo')
    f, = anim_ax.plot(football_coord.iloc[0]['h_x'],football_coord.iloc[0]['h_y'],'yo')

    def animate(i):
        h.set_data(home_coord.iloc[i]['h_x'],home_coord.iloc[i]['h_y'])
        a.set_data(away_coord.iloc[i]['h_x'],away_coord.iloc[i]['h_y'])
        f.set_data(football_coord.iloc[i]['h_x'],football_coord.iloc[i]['h_y'])

        return h,a,f,
    
    anim_ax.set_title(plays[plays['playId']==75].iloc[0]['playDescription'])
    animation_gif = animation.FuncAnimation(anim_fig,animate,frames=range(1,59),interval=85)
    
    return animation_gif